In [1]:
import codecs
import glob
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import cm

In [2]:
data_dir = "data/input/S-35C2/OH/A/dh6/*.out"
paths = [p for p in glob.glob(data_dir)]
initial_key = "low   system:  model energy"
end_key = "ONIOM: calculating electric field derivatives."
data_names = data_dir.split('/')[2:-1]
data_name = ""
for p in data_names:
    data_name = data_name + p
result_dir = "data/output/" + data_name + ".csv"
df_output = pd.DataFrame()

In [3]:
for path in paths:
    # file_nameから角度の情報を取る
    file_name = path.split('/')[-1]
    torsion_angle = int(file_name.split("@")[0].split("_")[-1])

    with codecs.open(path, "r", "UTF-8", "ignore") as file:
        df = pd.read_table(file)
        df = df.rename(columns={df.columns[0]: 'Col_1'})
        # initial_keyを含む行を探す
        initial_key_df = df[df['Col_1'].str.contains(initial_key, case=False)]
        initial_key_index = initial_key_df.index[0]
        df = df.iloc[initial_key_index:]
        # initial_keyを含む行以降のend_keyを含む行を抜き出す
        end_key_df = df[df['Col_1'].str.contains(end_key)]
        # dfを更新しているのでここでの行番号は0から始まる
        end_key_index = end_key_df.index[0] - initial_key_index
        df = df.iloc[:end_key_index]
        # 該当行の抜き出しが完了
        
        # 各エネルギー値の抜き出し
        low_model = float(list(df['Col_1'])[0].split(":")[-1])
        high_model = float(list(df['Col_1'])[1].split(":")[-1])
        low_real = float(list(df['Col_1'])[2].split(":")[-1])
        high_real = float(list(df['Col_1'])[3].split("=")[-1])
        
        df_E = pd.DataFrame(data=[high_real, low_real, high_model, low_model], 
                            columns=[torsion_angle],
                            index=["high real", "low real", "high model", "low model"]).T
        df_output = pd.concat([df_output, df_E])

df_output = df_output.sort_index()
df_output.to_csv(result_dir)
df_output

ValueError: could not convert string to float: ' ******************'

In [ ]:
df_output = df_output.sort_index()
df_output.to_csv(result_dir)

In [12]:
# low realが*****の場合
import codecs
import glob
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import cm

data_dir = "data/input/S-35C2/OH/A/dh6/*.out"
paths = [p for p in glob.glob(data_dir)]
initial_key = "low   system:  model energy"
end_key = "ONIOM: calculating electric field derivatives."
data_names = data_dir.split('/')[2:-1]
data_name = ""
for p in data_names:
    data_name = data_name + p
result_dir = "data/output/" + data_name + ".csv"
df_output = pd.DataFrame()

def l_real(h_real,h_model,l_model):
    return h_real - h_model + l_model

for path in paths:
    # file_nameから角度の情報を取る
    file_name = path.split('/')[-1]
    torsion_angle = int(file_name.split("@")[0].split("_")[-1])

    with codecs.open(path, "r", "UTF-8", "ignore") as file:
        df = pd.read_table(file)
        df = df.rename(columns={df.columns[0]: 'Col_1'})
        # initial_keyを含む行を探す
        initial_key_df = df[df['Col_1'].str.contains(initial_key, case=False)]
        initial_key_index = initial_key_df.index[0]
        df = df.iloc[initial_key_index:]
        # initial_keyを含む行以降のend_keyを含む行を抜き出す
        end_key_df = df[df['Col_1'].str.contains(end_key)]
        # dfを更新しているのでここでの行番号は0から始まる
        end_key_index = end_key_df.index[0] - initial_key_index
        df = df.iloc[:end_key_index]
        # 該当行の抜き出しが完了
        
        # 各エネルギー値の抜き出し
        low_model = float(list(df['Col_1'])[0].split(":")[-1])
        high_model = float(list(df['Col_1'])[1].split(":")[-1])
#         low_real = float(list(df['Col_1'])[2].split(":")[-1])
        high_real = float(list(df['Col_1'])[3].split("=")[-1])
        
        df_E = pd.DataFrame(data=[high_real, high_model, low_model], 
                            columns=[torsion_angle],
                            index=["high real", "high model", "low model"]).T
        
        df_output = pd.concat([df_output, df_E])

df_output['low real'] = df_output.apply(lambda f: l_real(f['high real'], f['high model'], f['low model']), axis=1)
df_output = df_output.sort_index()
df_output.to_csv(result_dir)
df_output


,high real,high model,low model,low real
-165,-27606.515342,-1630.004575,-1623.518090,-27600.028857
-150,-27606.515624,-1630.003931,-1623.517959,-27600.029652
-135,-27606.514282,-1630.001721,-1623.516330,-27600.028892
-120,-27606.512001,-1629.998392,-1623.513468,-27600.027077
-105,-27606.509308,-1629.994623,-1623.510083,-27600.024768
-90,-27606.508465,-1629.991752,-1623.507546,-27600.024259
-75,-27606.513720,-1629.994469,-1623.509445,-27600.028696
-60,-27606.524319,-1630.002175,-1623.515758,-27600.037903
-45,-27606.535010,-1630.010329,-1623.522752,-27600.047433
-30,-27606.542517,-1630.016442,-1623.528184,-27600.054259


In [13]:
df_output = df_output.sort_index()
df_output.to_csv(result_dir)

# ファイル名ごとにまとめたい場合

In [2]:
# low realが*****の場合
import codecs
import glob
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import cm

data_dir = "data/input/S-35C2/OH/A/dh6/*.out"
paths = [p for p in glob.glob(data_dir)]
initial_key = "low   system:  model energy"
end_key = "ONIOM: calculating electric field derivatives."
data_names = data_dir.split('/')[2:-1]
data_name = ""
for p in data_names:
    data_name = data_name + p
result_dir = "data/output/" + data_name + ".csv"
df_output = pd.DataFrame()

def l_real(h_real,h_model,l_model):
    return h_real - h_model + l_model

for path in paths:
    # file_nameから角度の情報を取る
    file_name = path.split('/')[-1]
    torsion_angle = int(file_name.split("@")[0].split("_")[-1])

    with codecs.open(path, "r", "UTF-8", "ignore") as file:
        df = pd.read_table(file)
        df = df.rename(columns={df.columns[0]: 'Col_1'})
        # initial_keyを含む行を探す
        initial_key_df = df[df['Col_1'].str.contains(initial_key, case=False)]
        initial_key_index = initial_key_df.index[0]
        df = df.iloc[initial_key_index:]
        # initial_keyを含む行以降のend_keyを含む行を抜き出す
        end_key_df = df[df['Col_1'].str.contains(end_key)]
        # dfを更新しているのでここでの行番号は0から始まる
        end_key_index = end_key_df.index[0] - initial_key_index
        df = df.iloc[:end_key_index]
        # 該当行の抜き出しが完了
        
        # 各エネルギー値の抜き出し
        low_model = float(list(df['Col_1'])[0].split(":")[-1])
        high_model = float(list(df['Col_1'])[1].split(":")[-1])
#         low_real = float(list(df['Col_1'])[2].split(":")[-1])
        high_real = float(list(df['Col_1'])[3].split("=")[-1])
        
        df_E = pd.DataFrame(data=[high_real, high_model, low_model], 
                            columns=[file_name],
                            index=["high real", "high model", "low model"]).T
        
        df_output = pd.concat([df_output, df_E])

df_output['low real'] = df_output.apply(lambda f: l_real(f['high real'], f['high model'], f['low model']), axis=1)
df_output = df_output.sort_index()
df_output.to_csv(result_dir)
df_output


,high real,high model,low model,low real
S-35Cl_A_-105@4_dh6.out,-27606.509308,-1629.994623,-1623.510083,-27600.024768
S-35Cl_A_-120@4_dh6.out,-27606.512001,-1629.998392,-1623.513468,-27600.027077
S-35Cl_A_-135@4_dh6.out,-27606.514282,-1630.001721,-1623.516330,-27600.028892
S-35Cl_A_-150@4_dh6.out,-27606.515624,-1630.003931,-1623.517959,-27600.029652
S-35Cl_A_-15@4_dh6.out,-27606.546998,-1630.020114,-1623.531480,-27600.058364
S-35Cl_A_-165@4_dh6.out,-27606.515342,-1630.004575,-1623.518090,-27600.028857
S-35Cl_A_-30@4_dh6.out,-27606.542517,-1630.016442,-1623.528184,-27600.054259
S-35Cl_A_-45@4_dh6.out,-27606.535010,-1630.010329,-1623.522752,-27600.047433
S-35Cl_A_-60@4_dh6.out,-27606.524319,-1630.002175,-1623.515758,-27600.037903
S-35Cl_A_-75@4_dh6.out,-27606.513720,-1629.994469,-1623.509445,-27600.028696
